# Populate & Modify Database

## 1. Populate database

Line `directory db` in `slapd.conf` instructs slapd to populate and write database to `db` directory.

In [1]:
# clean up db before populate
! rm -f db/* && ls -lh db

total 0


Validate `data.ldif` before populate it into database.

In [2]:
! ldapadd -v -n -f data.ldif

add objectclass:
	top
	organization
	dcobject
add o:
	Example, Inc.
add description:
	The Example, Inc. at Anytown.
!adding new entry "dc=example,dc=com"



Populate database.

In [3]:
! slapd -Ta -f slapd.conf -l data.ldif

Closing DB...


Check `db` directory.

In [4]:
! ls -lh db

total 36K
-rw-------. 1 root root  32K Apr 21 13:00 data.mdb
-rw-------. 1 root root 8.1K Apr 21 13:00 lock.mdb


## 2. Start SLAPD

Validate `slapd.conf` before starting SLAPD.

In [5]:
! slaptest -u -f slapd.conf

config file testing succeeded


Kill old SLAPD instances.

In [6]:
! killall slapd

Start SLAPD.

In [7]:
! slapd -f slapd.conf -h ldap://localhost:9011 

Check if SLAPD is running.

In [8]:
! lsof -P -i -n | grep slapd

slapd     131265            root    7u  IPv6 506575      0t0  TCP [::1]:9011 (LISTEN)
slapd     131265            root    8u  IPv4 506576      0t0  TCP 127.0.0.1:9011 (LISTEN)


## 3. Search

In [9]:
! ldapsearch -x -H ldap://localhost:9011 -LLL -b 'dc=example,dc=com'

dn: dc=example,dc=com
objectClass: top
objectClass: organization
objectClass: dcobject
o: Example, Inc.
description: The Example, Inc. at Anytown.
dc: example



## 4. Modify

Validate `modify.ldif` before applying it to database.

In [10]:
! ldapmodify -v -n -f modify.ldif

replace description:
	The Example, Inc. at Mytown.
!modifying entry "dc=example,dc=com"



Modify database.

In [11]:
! ldapmodify -x -H ldap://localhost:9011 -D cn=Manager,dc=example,dc=com -w secret -f modify.ldif

modifying entry "dc=example,dc=com"



Validate changes.

In [12]:
! ldapsearch -x -H ldap://localhost:9011 -LLL -b 'dc=example,dc=com'

dn: dc=example,dc=com
objectClass: top
objectClass: organization
objectClass: dcobject
o: Example, Inc.
dc: example
description: The Example, Inc. at Mytown.

